In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.geometry import Point
from shapely import wkb
from shapely import errors
import itertools
import matplotlib.pyplot as plt
import datetime as dt
import holidays
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


In [2]:
df = pd.read_parquet("/Users/michaelbrady/Documents/repo/DU-VIRT-AI-PT-11-2024-U-LOLC/ny_ride_fair_optimization/nik_folder/Resources/model_df.parquet")

In [3]:
pd.set_option('display.max_columns', None)
df

,"('second_of_day',)","('day_of_year',)","('weekend',)","('holiday',)","('morning_rush',)","('evening rush',)","('prcp',)","('temp',)","('PUx',)","('PUy',)","('DOx',)","('DOy',)","('distance',)","('duration(sec)',)","('fare',)","('tolls',)","('airport',)","('congestion',)","('total',)","('class',)","('PU_Bronx',)","('PU_Brooklyn',)","('PU_Manhattan',)","('PU_Queens',)","('PU_Staten Island',)","('DO_Bronx',)","('DO_Brooklyn',)","('DO_Manhattan',)","('DO_Queens',)","('DO_Staten Island',)"
0,5,1,False,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,-8.232220e+06,4.983524e+06,18.85,1599.0,70.00,6.94,0.0,0.0,95.88,0,False,False,False,True,False,False,False,True,False,False
1,9,1,False,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,-8.218525e+06,4.993771e+06,2.47,785.0,14.59,0.00,0.0,0.0,14.59,1,True,False,False,False,False,True,False,False,False,False
2,33,1,False,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,-8.232025e+06,4.972525e+06,2.02,474.0,20.62,0.00,0.0,0.0,20.62,2,False,False,False,True,False,False,True,False,False,False
3,38,1,False,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,-8.216190e+06,4.976140e+06,2.47,459.0,23.20,0.00,0.0,0.0,23.20,2,False,False,False,True,False,False,False,False,True,False
4,42,1,False,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,-8.235160e+06,4.976716e+06,1.06,335.0,7.90,0.00,0.0,2.5,15.48,0,False,False,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,False,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,-8.221850e+06,4.985513e+06,2.48,534.0,14.38,0.00,0.0,0.0,14.38,2,True,False,False,False,False,True,False,False,False,False
6044443,85864,366,False,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,-8.231941e+06,4.963280e+06,1.58,488.0,15.01,0.00,0.0,0.0,15.01,2,False,True,False,False,False,False,True,False,False,False
6044444,85870,366,False,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,-8.236077e+06,4.972248e+06,0.60,256.0,5.80,0.00,0.0,2.5,10.80,0,False,False,True,False,False,False,False,True,False,False
6044445,86042,366,False,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,-8.224081e+06,4.965819e+06,1.36,266.0,8.92,0.00,0.0,0.0,8.92,1,False,False,False,True,False,False,True,False,False,False


In [4]:
pd.set_option('display.max_columns', None)
df.columns = df.columns.str.replace(r"[()',]", "", regex=True).str.strip()
df = df.replace({True: 1, False: 0})

df.rename(columns={"fore": "fare"}, inplace=True)
df.rename(columns={"class": "service"}, inplace=True)
df

/var/folders/vt/jy8dcmxj29vfz95p7lls9h1m0000gn/T/ipykernel_88566/2756985136.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


,second_of_day,day_of_year,weekend,holiday,morning_rush,evening rush,prcp,temp,PUx,PUy,DOx,DOy,distance,durationsec,fare,tolls,airport,congestion,total,service,PU_Bronx,PU_Brooklyn,PU_Manhattan,PU_Queens,PU_Staten Island,DO_Bronx,DO_Brooklyn,DO_Manhattan,DO_Queens,DO_Staten Island
0,5,1,0,1,0,0,0.0,42.80,-8.213879e+06,4.960411e+06,-8.232220e+06,4.983524e+06,18.85,1599.0,70.00,6.94,0.0,0.0,95.88,0,0,0,0,1,0,0,0,1,0,0
1,9,1,0,1,0,0,0.0,42.80,-8.220657e+06,4.992372e+06,-8.218525e+06,4.993771e+06,2.47,785.0,14.59,0.00,0.0,0.0,14.59,1,1,0,0,0,0,1,0,0,0,0
2,33,1,0,1,0,0,0.0,42.80,-8.231953e+06,4.974857e+06,-8.232025e+06,4.972525e+06,2.02,474.0,20.62,0.00,0.0,0.0,20.62,2,0,0,0,1,0,0,1,0,0,0
3,38,1,0,1,0,0,0.0,42.80,-8.212420e+06,4.977409e+06,-8.216190e+06,4.976140e+06,2.47,459.0,23.20,0.00,0.0,0.0,23.20,2,0,0,0,1,0,0,0,0,1,0
4,42,1,0,1,0,0,0.0,42.80,-8.235990e+06,4.975327e+06,-8.235160e+06,4.976716e+06,1.06,335.0,7.90,0.00,0.0,2.5,15.48,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6044442,85846,366,0,0,0,0,0.0,48.02,-8.217649e+06,4.988984e+06,-8.221850e+06,4.985513e+06,2.48,534.0,14.38,0.00,0.0,0.0,14.38,2,1,0,0,0,0,1,0,0,0,0
6044443,85864,366,0,0,0,0,0.0,48.02,-8.234034e+06,4.964909e+06,-8.231941e+06,4.963280e+06,1.58,488.0,15.01,0.00,0.0,0.0,15.01,2,0,1,0,0,0,0,1,0,0,0
6044444,85870,366,0,0,0,0,0.0,48.02,-8.236077e+06,4.972248e+06,-8.236077e+06,4.972248e+06,0.60,256.0,5.80,0.00,0.0,2.5,10.80,0,0,0,1,0,0,0,0,1,0,0
6044445,86042,366,0,0,0,0,0.0,48.02,-8.220615e+06,4.964609e+06,-8.224081e+06,4.965819e+06,1.36,266.0,8.92,0.00,0.0,0.0,8.92,1,0,0,0,1,0,0,1,0,0,0


In [5]:
df["service"].value_counts()

service
2    2074749
1    2019704
0    1949994
Name: count, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [7]:
df_taxi = df[(df["service"] == 0) & (df["fare"] > 0)]
df_uber = df[(df["service"] == 1) & (df["fare"] > 0)]
df_lyft = df[(df["service"] == 2) & (df["fare"] > 0)]

# removing total and duration and for leakage

features = ["second_of_day", "day_of_year", "PUx", "PUy", "DOx", "DOy", "distance", "morning_rush", "evening rush",
            "prcp", "temp", "holiday", "weekend", "airport", "congestion", "PU_Bronx", "PU_Brooklyn", 
            "PU_Manhattan", "PU_Queens", "PU_Staten Island", "DO_Bronx", "DO_Brooklyn", 
            "DO_Manhattan", "DO_Queens", "DO_Staten Island"]

In [8]:
"""
taxi_model = LinearRegression().fit(df_taxi[features], df_taxi["fare"])
uber_model = LinearRegression().fit(df_uber[features], df_uber["fare"])
lyft_model = LinearRegression().fit(df_lyft[features], df_lyft["fare"])
"""

'\ntaxi_model = LinearRegression().fit(df_taxi[features], df_taxi["fare"])\nuber_model = LinearRegression().fit(df_uber[features], df_uber["fare"])\nlyft_model = LinearRegression().fit(df_lyft[features], df_lyft["fare"])\n'

In [9]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# final NaN check
df = df.dropna()

y = df_taxi["fare"] 
X = df_taxi[features]

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV


In [10]:

param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
}

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb:.2f}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)

Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}
XGBoost MAE: 2.19
R² via r2_score: 0.9112410259263726


In [11]:
df_taxi["fare"].describe()

count    1.949613e+06
mean     1.950895e+01
std      1.709667e+01
min      1.000000e-02
25%      9.300000e+00
50%      1.350000e+01
75%      2.190000e+01
max      5.000000e+02
Name: fare, dtype: float64

In [12]:
y = df_uber["fare"] 
X = df_uber[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb}")
r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)



Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}
XGBoost MAE: 4.403072200941467
R² via r2_score: 0.8152788394705536


In [13]:
y = df_lyft["fare"] 
X = df_lyft[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

random_search = RandomizedSearchCV(xgb.XGBRegressor(random_state=42), param_distributions=param_dist, n_iter=10, cv=3, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
y_pred_xgb = random_search.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print(f"XGBoost MAE: {mae_xgb}")

r2_via_metric = r2_score(y_test, y_pred_xgb)
print("R² via r2_score:", r2_via_metric)


Best Parameters: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.6}
XGBoost MAE: 3.1377697317875075
R² via r2_score: 0.8832726185987507


In [14]:
#import statsmodels.api as sm

#X = df_uber[["second_of_day", "day_of_year", "PUx", "PUy", "DOx", "DOy", "distance",
            "holiday", "weekend", "airport", "congestion", "PU_Bronx", "PU_Brooklyn", 
            "PU_Manhattan", "PU_Queens", "PU_Staten Island", "DO_Bronx", "DO_Brooklyn", 
            "DO_Manhattan", "DO_Queens", "DO_Staten Island"]]
y = df_uber["fare"]

# Add a constant (intercept term)
X = sm.add_constant(X)

# Fit the Ordinary Least Squares (OLS) regression model
model = sm.OLS(y, X).fit()

# Print summary with p-values
print(model.summary())


IndentationError: unexpected indent (1615157829.py, line 4)

In [ ]:
#import joblib

#joblib.dump(model, "log_regres_first.pkl")


In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled)
X_train_scaled_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,1.051171,0.088964,2.217391,-1.335392,2.277855,-0.922203,-0.051240,-0.36227,-0.350328,-0.156567,-0.691846,3.280351,-0.809238,-0.337809,-0.623886,-0.784657,1.879444,-0.126249,-0.33845,-0.620522,-0.755467,1.814570,-0.128143
1,-0.223320,-0.875721,0.499113,-0.148283,0.247113,-0.489164,-0.724996,-0.36227,-0.350328,-0.156567,-0.691846,-0.299491,-0.809238,-0.337809,-0.623886,-0.784657,1.879444,-0.126249,-0.33845,1.611546,-0.755467,-0.551095,-0.128143
2,0.608264,1.472287,0.381457,-1.073193,-0.037203,-1.345430,-0.727257,-0.36227,-0.350328,-0.156567,1.445407,-0.299491,-0.809238,-0.337809,1.602858,-0.784657,-0.532072,-0.126249,-0.33845,1.611546,-0.755467,-0.551095,-0.128143
3,-1.422917,1.599698,1.728814,-0.885920,1.626222,-0.860299,-0.636820,-0.36227,-0.350328,-0.156567,1.445407,-0.299491,-0.809238,-0.337809,-0.623886,-0.784657,1.879444,-0.126249,-0.33845,-0.620522,-0.755467,1.814570,-0.128143
4,-0.050649,-1.567383,1.310208,-0.187803,1.010314,-0.098604,-0.639081,-0.36227,-0.350328,-0.156567,-0.691846,-0.299491,-0.809238,-0.337809,-0.623886,-0.784657,1.879444,-0.126249,-0.33845,-0.620522,-0.755467,1.814570,-0.128143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658770,0.802853,1.008146,-0.577504,-0.660368,-0.452972,-0.837752,-0.749866,-0.36227,2.854466,-0.156567,-0.691846,-0.299491,-0.809238,-0.337809,1.602858,-0.784657,-0.532072,-0.126249,-0.33845,1.611546,-0.755467,-0.551095,-0.128143
1658771,0.389115,-0.966730,-0.170313,-1.154231,-0.170202,-1.125384,-0.883261,-0.36227,2.854466,-0.156567,-0.691846,-0.299491,-0.809238,-0.337809,1.602858,-0.784657,-0.532072,-0.126249,-0.33845,1.611546,-0.755467,-0.551095,-0.128143
1658772,0.785460,-0.857520,-1.188826,-0.332695,-0.851179,-0.813940,-0.184635,-0.36227,2.854466,-0.156567,-0.691846,-0.299491,1.226738,-0.337809,-0.623886,1.274443,-0.532072,-0.126249,-0.33845,1.611546,-0.755467,-0.551095,-0.128143
1658773,0.637475,1.262968,-1.067155,-0.244259,-0.686111,0.414502,0.043719,-0.36227,2.854466,-0.156567,-0.691846,-0.299491,1.226738,-0.337809,-0.623886,1.274443,-0.532072,-0.126249,-0.33845,-0.620522,1.323684,-0.551095,-0.128143


In [ ]:

feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print(feature_importance)


AttributeError: 'OLSResults' object has no attribute 'coef_'

In [ ]:
y_proba = model.predict_proba(X_test_scaled)
print(y_proba[:5])  # First 5 rows


[[9.99838609e-01 1.08419800e-04 5.29712970e-05]
 [9.95477590e-01 1.96036635e-03 2.56204318e-03]
 [9.99998085e-01 1.67485476e-06 2.40040660e-07]
 [9.99652864e-01 2.23916385e-04 1.23219497e-04]
 [9.99651130e-01 2.42814012e-04 1.06055707e-04]]


In [ ]:
y_train.value_counts()

class
0    19581462
2     1660328
1      533913
Name: count, dtype: int64

In [ ]:
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error, mean_squared_error


# Select features and target
features = ["second_of_day", "PUx", "PUy", "DOx", "DOy", "distance", "durationsec", 
            "holiday", "weekend", "airport", "congestion", "PU_Bronx", "PU_Brooklyn", 
            "PU_Manhattan", "PU_Queens", "PU_Staten Island", "DO_Bronx", "DO_Brooklyn", 
            "DO_Manhattan", "DO_Queens", "DO_Staten Island"]

# Target variable (Fare price)
target = "fore"

# Split the data into train and test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Check if dataset has missing values
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)


In [ ]:
# Initialize the Linear Regression Model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Predict fares
y_pred = model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")


Mean Absolute Error (MAE): $3.61
Root Mean Squared Error (RMSE): $7.09


In [15]:
# Example new trip (Replace with actual values)
new_trip = pd.DataFrame([{
    "second_of_day": 36000,  # 10:00 AM
    "PUx": 40.7, "PUy": -74,  # Pickup coordinates
    "DOx": 40.8, "DOy": -73.9,  # Dropoff coordinates
    "distance": 3.5,  # Miles
    "durationsec": 600,  # 10 minutes
    "holiday": 0, "weekend": 1, "airport": 0, "congestion": 2,
    "PU_Bronx": 0, "PU_Brooklyn": 0, "PU_Manhattan": 1, "PU_Queens": 0, "PU_Staten Island": 0,
    "DO_Bronx": 0, "DO_Brooklyn": 0, "DO_Manhattan": 1, "DO_Queens": 0, "DO_Staten Island": 0
}])

# Predict the fare
predicted_fare = model.predict(new_trip)
print(f"Predicted Fare: ${predicted_fare[0]:.2f}")


NameError: name 'model' is not defined

In [ ]:
# fix class imbalance

#undersampler = RandomUnderSampler(sampling_strategy="auto", random_state=42)

#X_train, y_train = undersampler.fit_resample(X_train, y_train)